# Workshop 4: errors

## Esercizio 1 
Si consideri l'equazione quadratica $x^2 + 2px - q = 0 $ con $p=10^5$ e $q=10^{-i}$, i=0,..,10

- Si stabilisca se il problema relativo al calcolo della soluzione $x= -p+\sqrt{p^2+q}$ risulta essere ben
      condizionato per tutti i valori di q assegnati;

- Si calcoli la soluzione x dell'equazione mediante la formula risolutiva indicata al punto precedente e si dica se l'algoritmo di calcolo risulta numericamente
stabile  per i valori di q assegnati

- si individui una forma stabile per il calcolo della soluzione e si verifichi se l'algoritmo di calcolo risulta numericamente stabile per i valori di q assegnati.

In [ ]:
import numpy as np

i = np.arange(0, 11, 1)
p = 10**5
q = 1 / 10**i

print("UNSTABLE: ", -p + np.sqrt(p**2 + q))
print("STABLE: ", q / (p + np.sqrt(p**2+q)))

# Esercizio 2

Si stabilisca se il problema relativo al calcolo della funzione $f(x)= \frac  {(1+x)-1 }  {x }$ per $x= 10^{-i }$ $  i=1,2,...,20$  risulta essere ben
condizionato per tutti i valori di x assegnati; verificare sperimentalmente la stabilità numerica dell'algoritmo di valutazione

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

f = lambda x: ((1 + x) - 1) / x
x = 1 / 10**(np.arange(1, 21))

print(f(x))
plt.semilogx(x, f(x), x,np.ones_like(x))


## Esercizio 3

Realizzate un codice python che implementa il seguente algoritmo. Si tratta dell'implementazione del  metodo di Archimede per l'approssimazione di pi-greco e genera una successione di valori $p_i$, i=1,n, (n scelto dall'utente) che tende a $\pi$. Verificare se all'aumentare di n, la successione $p_i$ tende a $\pi$.

b[0]=2

s[0]=1

for i in range(1,n):

    p[i]=b[i-1]*s[i-1]
    b[i]=2*b[i-1]
     s[i]=math.sqrt((1.0-math.sqrt(1-s[i-1]**2))/2.0)

- Visualizzate su un grafico gli elementi della successione p[i] calcolata.
- Calcolate e visualizzate su un grafico l'errore relativo per ogni componente di p[i], - considerando come valore esatto di pigreco il valore math.pi.

- Giustificate i risultati alla luce della teoria.

- Proponete una versione stabile dell'algoritmo

- Calcolate e visualizzate su un grafico l'errore relativo per ogni componente di p[i] prodotta dall'algoritmo stabilizzato, considerando come valore esatto di
pigreco il valore math.pi.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n = 34
b = np.zeros(n)
s = np.zeros(n)
p = np.zeros(n)

b[0] = 2
s[0] = 1

for i in range(1, n):
    p[i] = b[i - 1] * s[i - 1]
    b[i] = 2 * b[i - 1]
    s[i] = np.sqrt((1.0 - np.sqrt(1 - s[i - 1]**2)) / 2.0)

bs=np.zeros(n)
ss=np.zeros(n)
ps=np.zeros(n)

bs[0]=2
ss[0]=1
for i in range(1, n):
    ps[i] = bs[i - 1] * ss[i - 1]
    bs[i] = 2 * bs[i - 1]
    ss[i] = ss[i - 1] / np.sqrt((2.0 * (1.0 + np.sqrt(1.0 - ss[i - 1]**2))))

plt.plot(np.arange(1,n),p[1:n],np.arange(1,n),ps[1:n])

plt.show()

## Esercizio 4   
Si consideri lo sviluppo in serie dell'esponenziale $e^x=1+x+\frac{1}{2!}x^2+\frac{1}{3!}x^3+\cdots =\sum_{k=0}^\infty \frac{x^k}{k!} $. Si calcoli un'approssimazione dell'esponenziale troncando ad N=100 termini la serie, per valori di x che variano da -10 a 10 incluso. Si calcoli  per ogni valore di x l'errore relativo del valore approssimato rispetto al valore esatto, ottenuto mediante la funzione exp della libreria numpy. Cosa si osserva? Pensare ad un algoritmo più stabile ricordando che $e^{-x}=\frac{1}{e^{|x|}}$.



In [ ]:
import numpy as np

N = 100

## Esercizio 5
Calcolare l'approssimazione della derivata prima di f(x) = sin(x) in x = 1
mediante l'algoritmo del rapporto incrementale (f(x + h) - f(x))/h per valori decrescenti
di h, confrontandolo con il valore fornito dalla funzione di libreria per f'(x)
mediante calcolo dell'errore relativo.

In [ ]:
import numpy as np

k=np.arange(0,-21.0,-1)
h=10.0**k
x = 1

f = lambda x: np.sin(x)
df = lambda x: np.cos(x)
i = lambda x, h: (f(x + h) - f(x)) / h

e = np.abs(df(x) - i(x, h)) / np.abs(df(x))

print("f(x): ", f(x))
print("\nf'(x): ", df(x)) 
print("\nlim h->0 ", i(x, h))
print("\ne rel: ", e)

## Esercizio 6
Valutazione di un polinomio in un vettore di punti
-   Realizzare una function  che mediante lo schema di Horner  valuti un polinomio in un vettore di punti.
-   Realizzare una function  che utilizzi l’algoritmo classico per valutare un polinomio in un vettore di punti.
-   Valutare   i seguenti polinomi, con i due metodi implementati nei punti precedenti,  sul vettore di punti specificato  e  rappresentarli graficamente. Confrontarne i tempi 

         p(x)=x^8-16x^7+112x^6-448x^5+1120x^4-1792x^3+1792x^2-1024x+256  da valutare su vettore di  1000000 punti equidistanti in [1.8,2.2]
         p(x)= x^11 –8 x^9 +28 x^6 –7 x^4 +12 x^3 –4 x +1   da valutare su vettore di 1000000  punti in [-5:5]
         
Cosa conferma questo esperimento?


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def alg(a, x):
    p = a[0] * np.ones_like(x)
    r = 1
    for i in range(1, a.shape[0]):
        r = r * x
        p += a[i] * r
    return p

def horner(a, x):
    n = a.shape[0]
    p = a[n - 1] * np.ones_like(x)
    for i in range(2, n + 1):
        p = p * x + a[n - i] 
    return p

n = 1000000

p1 = lambda x: x**8 - 16 * x**7 + 112 * x**6 - 448 * x**5 + 1120 * x**4 - 1792 * x ** 3 + 1792 * x**2 - 1024 * x + 256
a1 = np.array([256, -1024, 1792, -1792, 1120, -448, 112, -16 ,1], dtype=float);
x1 = np.linspace(1.8, 2.2, n)

p2 = lambda x: x**11 - 8 * x**9 + 28 * x**6 - 7 * x**4 + 12  * x**3 - 4 * x + 1
x2 = np.linspace(-5, 5, n)

plt.plot(x1, alg(a1, x1), x1, horner(a1,x1))
plt.show()

## Esercizio 7
Realizzare un codice python che calcola il valore del polinomio p(x)=(x-2)^8 utilizzando le formule

    p=x^8-16x^7+112x^6-448x^5+1120x^4-1792x^3+1792x^2-1024x+256;
    p(x)=(x-2)^8
    e lo schema di Horner

e ne realizza il grafico in un vettore di 10000 equidistanti nell’intervallo [1.8,2.2], [1.9, 2.1],  [1.92,2.08], [1.93,2.07], [1.94,2.06] Spiegare il comportamento osservato. Cosa succede al restringersi dell’intervallo intorno al valore 2, punto in cui questo polinomio si annulla?




Si nota che al restringersi dell'intervallo in torno al punto x=2, 
in cui la funzione si annulla, si evidenzia l'instabilità dell'algoritmo 1
rispetto all' algoritmo di Horner che risulta più stabile. 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

f = lambda x: (x - 2.0)**8

n = 10000
x1 = np.linspace(1.8, 2.2, n)
x2 = np.linspace(1.9, 2.1, n)
x3 = np.linspace(1.92, 2.08, n)
x4 = np.linspace(1.93, 2.07, n)
x5 = np.linspace(1.94, 2.06, n)

plt.plot(x1, f(x1), x1, alg(a1, x1), x1, horner(a1, x1))
plt.legend(['P','P_1','P_H'])
plt.show()

plt.plot(x2, f(x2), x2, alg(a1, x2), x2, horner(a1, x2))
plt.legend(['P','P_1','P_H'])
plt.show()

plt.plot(x3, f(x3), x3, alg(a1, x3), x3, horner(a1, x3))
plt.legend(['P','P_1','P_H'])
plt.show()

plt.plot(x4, f(x4), x4, alg(a1, x4), x4, horner(a1, x4))
plt.legend(['P','P_1','P_H'])
plt.show()

plt.plot(x4, f(x4), x4, alg(a1, x4), x4, horner(a1, x4))
plt.legend(['P','P_1','P_H'])
plt.show()